In [173]:
import pandas

from datetime import datetime

from pandas import Timestamp

from pandas.tseries.offsets import DateOffset
# import zip



rateios_celesc = pandas.read_csv(r"C:\Users\Luan - PLIN\Documents\CELESC\COMPILADO-CELESC\COMPILADO-CELESC\COMPILADO RATEIOS - CELESC.csv")

rateios_celesc.drop(["OBSERVAÇÃO"],axis=1, inplace=True)

rateios_celesc['PORCENTAGEM']=rateios_celesc['PORCENTAGEM'].apply(lambda x : float(x.replace(",",".").replace("%",""))/100)

rateios_celesc['BENEFICIÁRIA']=rateios_celesc['BENEFICIÁRIA'].astype(str)

rateios_celesc['GERADORA']=rateios_celesc['GERADORA'].apply(str)

rateios_celesc["datetime"]=rateios_celesc["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

injecao_geradores = pandas.read_csv(r"C:\Users\Luan - PLIN\Documents\CELESC\COMPILADO-CELESC\COMPILADO-CELESC\Faturas PLIN - CELESC.csv")

injecao_geradores=injecao_geradores.rename({"ucGeradora":"GERADORA","referencia":"REFERÊNCIA"},axis=1)

columns_maintain = ["GERADORA", "REFERÊNCIA", "INJEÇÃO TP"]

injecao_geradores = injecao_geradores[columns_maintain]

injecao_geradores["INJEÇÃO TP"]=injecao_geradores["INJEÇÃO TP"].apply(lambda x : float(x)) 

injecao_geradores["GERADORA"]=injecao_geradores["GERADORA"].apply(str) 

injecao_geradores["datetime"]=injecao_geradores["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

leituras_celesc = pandas.read_csv(r"C:\Users\Luan - PLIN\Documents\CELESC\COMPILADO-CELESC\COMPILADO-CELESC\LEITURAS__Geradorasdeenergia_CELESC_Conta Corrente_Ativo_Erro de leitura_Vencidas_05-08-2024.csv")

leituras_celesc=leituras_celesc.rename({"uc":"UC","date_ref":"REFERÊNCIA","kwh_compensado":"COMPENSADO","kwh_consumed":"CONSUMIDO"},axis=1)

leituras_celesc["COMPENSADO"]=leituras_celesc["COMPENSADO"].apply(lambda x : float(abs(float(x))))

columns_maintain=["UC","REFERÊNCIA","COMPENSADO","CONSUMIDO"]

# TRATAMENTO DE DADOS LEITURAS

leituras_celesc=leituras_celesc[columns_maintain]

leituras_celesc["UC"]=leituras_celesc["UC"].apply(str)

leituras_celesc["REFERÊNCIA"]=leituras_celesc["REFERÊNCIA"].apply(lambda x: "01/"+x)

leituras_celesc["datetime"]=leituras_celesc["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

rateios_celesc=rateios_celesc.set_index(["GERADORA","datetime","REFERÊNCIA"])

injecao_geradores=injecao_geradores.set_index(["GERADORA","datetime","REFERÊNCIA"])

leituras_celesc.set_index(['UC',"datetime","REFERÊNCIA"],inplace=True)

# INSERÇÃO COLUNA INJEÇÃO CLIENTE USINA

def injecao_cliente_usina(row):
    try:
        inj_usina = injecao_geradores["INJEÇÃO TP"].loc[row.name].values
        inj_cliente_usina = inj_usina * row["PORCENTAGEM"]
        return float(format(inj_cliente_usina[0],".0f"))
    
    except:
        return None
    
rateios_celesc["INJ_CLIENTE_USINA"] = rateios_celesc.apply(injecao_cliente_usina,axis=1)

rateios_celesc.sort_index()

leituras_celesc.sort_index()

injecao_geradores.sort_index()


def dict_leitura (row:pandas.Series):
    # print(row.name)
    try:
        usinas_dict = rateios_celesc.query("BENEFICIÁRIA  == '{uc}' & REFERÊNCIA == '{ref}'".format(uc=row.name[0],ref = row.name[2]))["INJ_CLIENTE_USINA"].droplevel('REFERÊNCIA').to_dict()
        return usinas_dict
    except: 
        # new_row = pandas.DataFrame({(row.name[0],row.name[1]-DateOffset(months=1),datetime.strftime(datetime.strptime(row.name[2],"%d/%m/%Y")-DateOffset(months=1),"%d/%m/%Y")):[0]})
        return 0    

leituras_celesc["INJEÇÃO_USINAS"] = leituras_celesc.apply(dict_leitura,axis=1)

def inj_total (row): 

    return sum(row[2].values())

leituras_celesc["INJ_TOTAL"] = leituras_celesc.apply(inj_total,axis=1)

leituras_celesc=leituras_celesc.drop(["CONSUMIDO"],axis=1)

leituras_celesc=leituras_celesc.sort_index(axis=0,level='datetime',ascending=True)

def compensado(row):

    saldo_anterior_total = row


C:\Users\Luan - PLIN\AppData\Local\Temp\ipykernel_6252\4293618897.py:66: PerformanceWarning: indexing past lexsort depth may impact performance.
  inj_usina = injecao_geradores["INJEÇÃO TP"].loc[row.name].values


INJEÇÃO TP
GERADORA datetime   REFERÊNCIA            
18247410 2024-03-01 01/03/2024      3912.0
         2024-04-01 01/04/2024      2556.0
         2024-05-01 01/05/2024      3234.0
         2024-06-01 01/06/2024       796.0
43155997 2024-03-01 01/03/2024      6616.0
...                                    ...
58307858 2024-06-01 01/06/2024         0.0
58355356 2024-03-01 01/03/2024      7616.0
         2024-04-01 01/04/2024      4607.0
         2024-05-01 01/05/2024      6111.0
         2024-06-01 01/06/2024       617.0

[70 rows x 1 columns]

In [174]:

leituras_celesc.query("UC  == '{beneficiaria}' & REFERÊNCIA == '{ref}'".format(uc=leituras_celesc,ref = row.name[2]))["INJ_CLIENTE_USINA"].droplevel('REFERÊNCIA')
    

,,,COMPENSADO,CONSUMIDO
UC,datetime,REFERÊNCIA,,
44695073,2024-07-01,01/07/2024,343.0,393.0
30937376,2024-07-01,01/07/2024,3206.0,3236.0
45662268,2024-07-01,01/07/2024,1520.0,1550.0
21707406,2024-07-01,01/07/2024,4661.0,4761.0
42715239,2024-07-01,01/07/2024,1362.0,1412.0
...,...,...,...,...
43808907,2023-04-01,01/04/2023,902.0,1470.0
51033795,2023-04-01,01/04/2023,239.0,289.0
28842813,2023-04-01,01/04/2023,1141.0,2248.0


In [22]:
import json
import pandas
import numpy

with open(r'C:\Users\Luan - PLIN\Downloads\all.json', 'r') as f:
    compilado_faturas=json.load(f)

compilado_faturas= pandas.DataFrame(compilado_faturas).fillna(0)
pandas.DataFrame([i for i in compilado_faturas.values.flatten() if i != 0]
).to_excel(r'compilado_histórico_faturas.xlsx')
